In [26]:
import json
import re
import pandas as pd
import nltk
import math
from math import sqrt
import sklearn
import statsmodels.api as sm
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
import re
import csv
import numpy as np

In [27]:
#Read review_train.json file
rv_train = []
with open('review_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    rv_tra = json.loads(line)
                    rv_train.append(rv_tra)
                else:
                    break
        except:
            f.close()

In [28]:
#Read business_train.json file
bs_train = []
with open('business_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    bs_tra = json.loads(line)
                    bs_train.append(bs_tra)
                    #print(bs_train)
                else:
                    break
        except:
            f.close()

In [29]:
bs_train = pd.DataFrame(bs_train)

In [30]:
category = bs_train[["categories"]]
pat_rest = re.compile(r'.*Gym.*')
num_no_rest = []
for row in tqdm(range(len(category))):
    if category.iloc[row]['categories'] is None:
        num_no_rest.append(row)
    else:
        if not pat_rest.match(category.iloc[row]['categories']):
            num_no_rest.append(row)
bs_train = bs_train.drop(index = num_no_rest)

100%|██████████| 154606/154606 [00:28<00:00, 5441.16it/s]


In [31]:
bs_train

,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,state
16,"{'AcceptsInsurance': 'True', 'ByAppointmentOnl...",104170,"Trainers, Health & Medical, Active Life, Physi...",Toronto,"{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ...",1,43.645041,-79.395799,Totum Life Science,M5V 1K4,ON
397,{'BusinessAcceptsCreditCards': 'True'},49708,"Active Life, Trainers, Fitness & Instruction, ...",North Las Vegas,"{'Monday': '15:30-19:30', 'Tuesday': '15:30-19...",1,36.276098,-115.203534,Iron Talon Crossfit,89084,NV
441,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...",173815,"Gyms, Fitness & Instruction, Active Life",Glendale,"{'Monday': '17:30-21:0', 'Tuesday': '17:30-21:...",1,33.520077,-112.184332,Relentless Power Gym,85301,AZ
494,"{'GoodForKids': 'False', 'ByAppointmentOnly': ...",183154,"Fitness & Instruction, Gyms, Active Life",Toronto,"{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ...",1,43.638122,-79.418156,F45 Training Liberty Village,M6K 3E7,ON
568,"{'BusinessParking': '{'garage': False, 'street...",138163,"Gyms, Active Life, Fitness & Instruction, Card...",Verdun,None,1,45.465383,-73.566886,Energie Cardio,H4G 1V1,QC
622,"{'GoodForKids': 'False', 'BusinessParking': '{...",17594,"Active Life, Cardio Classes, Fitness & Instruc...",Las Vegas,"{'Monday': '4:30-21:30', 'Tuesday': '4:30-20:3...",1,36.298839,-115.279247,Orange Theory Fitness Las Vegas - Centennial,89117,NV
681,"{'BikeParking': 'False', 'ByAppointmentOnly': ...",106407,"Taekwondo, Gyms, Martial Arts, Education, Acti...",Henderson,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,36.042930,-115.116226,WTM Taekwondo,89074,NV
710,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...",170000,"Sports Clubs, Local Services, Community Servic...",Charlotte,"{'Monday': '5:0-21:30', 'Tuesday': '5:0-21:30'...",1,35.315808,-80.778680,University City Branch YMCA,28262,NC
713,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",69171,"Active Life, Gyms, Trainers, Fitness & Instruc...",Charlotte,"{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ...",1,35.101964,-80.991144,Fitness Together,28278,NC
801,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",22591,"Gyms, Trainers, Active Life, Fitness & Instruc...",Henderson,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.035840,-115.098688,24 Hour Fitness - Wigwam Henderson,89074,NV


In [32]:
bs_train.columns

Index(['attributes', 'business_id', 'categories', 'city', 'hours', 'is_open',
       'latitude', 'longitude', 'name', 'postal_code', 'state'],
      dtype='object')

In [33]:
bs_attr = list(dict(bs_train['attributes']).values())

In [38]:
attr = pd.DataFrame(bs_attr[0], index = [0])
for i in range(1,len(bs_attr)):
    attr = pd.concat([attr,pd.DataFrame(bs_attr[i],index = [i] )])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
bs_parking = list(dict(attr['BusinessParking']).values())
parking = pd.DataFrame(np.nan, index = [0,1], columns = ['garage','street','validated','lot', 'valet'])
for i in range(2,len(bs_parking)):
    if pd.isnull(bs_parking[i]) == True:
        parking.loc[i] = np.nan
    else:
        parking = pd.concat([parking,pd.DataFrame(eval(bs_parking[i]), index = [i])])
        parking[i:(i+1)] = (parking[i:(i+1)] == 1.0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [43]:
#Add the attributes as columns of business_train.
bs_train = bs_train.drop(['attributes'],axis=1)
bs_train = bs_train.reset_index(drop=True)
business_train = pd.concat([bs_train,attr],axis=1)
business_train.to_csv("business_train.csv")

In [49]:
#Merge the columns of attributes of parking and other attributes, we get a 1886*30 dataframe.
attr=pd.concat([attr,parking],axis = 1)  

In [51]:
#Drop the attributes with missing value more than 35%
N = len(attr)*0.35
names = attr.columns
for i in range(len(names)):
    if sum(pd.isnull(attr[names[i]])) > N:
        attr = attr.drop(names[i],axis = 1)
attr = attr.drop(['BusinessParking'],axis=1)

In [52]:
#Split the parking attributes as columns of bussiness_train.
bs_train_pk = pd.concat([bs_train,attr],axis=1)

In [53]:
#Write the dataframe business_train as csv file, which is merged with BusinessParking splited attribures.
bs_train_pk.to_csv("bs_train_pk.csv")

In [54]:
bs_train_pk

,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,state,ByAppointmentOnly,GoodForKids,garage,lot,street,valet,validated
0,104170,"Trainers, Health & Medical, Active Life, Physi...",Toronto,"{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ...",1,43.645041,-79.395799,Totum Life Science,M5V 1K4,ON,False,False,NaN,NaN,NaN,NaN,NaN
1,49708,"Active Life, Trainers, Fitness & Instruction, ...",North Las Vegas,"{'Monday': '15:30-19:30', 'Tuesday': '15:30-19...",1,36.276098,-115.203534,Iron Talon Crossfit,89084,NV,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,173815,"Gyms, Fitness & Instruction, Active Life",Glendale,"{'Monday': '17:30-21:0', 'Tuesday': '17:30-21:...",1,33.520077,-112.184332,Relentless Power Gym,85301,AZ,False,True,False,False,False,False,False
3,183154,"Fitness & Instruction, Gyms, Active Life",Toronto,"{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ...",1,43.638122,-79.418156,F45 Training Liberty Village,M6K 3E7,ON,False,False,NaN,NaN,NaN,NaN,NaN
4,138163,"Gyms, Active Life, Fitness & Instruction, Card...",Verdun,None,1,45.465383,-73.566886,Energie Cardio,H4G 1V1,QC,NaN,False,False,False,False,False,False
5,17594,"Active Life, Cardio Classes, Fitness & Instruc...",Las Vegas,"{'Monday': '4:30-21:30', 'Tuesday': '4:30-20:3...",1,36.298839,-115.279247,Orange Theory Fitness Las Vegas - Centennial,89117,NV,True,False,False,False,False,False,False
6,106407,"Taekwondo, Gyms, Martial Arts, Education, Acti...",Henderson,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,36.042930,-115.116226,WTM Taekwondo,89074,NV,False,NaN,False,True,False,False,False
7,170000,"Sports Clubs, Local Services, Community Servic...",Charlotte,"{'Monday': '5:0-21:30', 'Tuesday': '5:0-21:30'...",1,35.315808,-80.778680,University City Branch YMCA,28262,NC,False,True,False,False,False,False,False
8,69171,"Active Life, Gyms, Trainers, Fitness & Instruc...",Charlotte,"{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ...",1,35.101964,-80.991144,Fitness Together,28278,NC,True,NaN,NaN,NaN,NaN,NaN,NaN
9,22591,"Gyms, Trainers, Active Life, Fitness & Instruc...",Henderson,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.035840,-115.098688,24 Hour Fitness - Wigwam Henderson,89074,NV,False,True,False,False,False,False,False
